In [1]:
!python -V

Python 3.5.6 :: Anaconda, Inc.


In [2]:
!pip install kfp --upgrade


  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 112kB 2.7MB/s 
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 122kB 3.6MB/s 
Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 163kB 3.8MB/s 
Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 235kB 4.4MB/s 
  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 266kB 3.7MB/s 
  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 81kB 6.9MB/s 
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
    100% |███████████████████

In [3]:
import kfp
from kfp import gcp
from kfp import dsl

import os
from datetime import datetime
import kfp.compiler as compiler

client = kfp.Client(host='511b4fa793007e03-dot-us-central1.notebooks.googleusercontent.com')

In [4]:
# Set your output and project
YOUR_NAME = 'erwinh' #change to your name
EXPERIMENT_NAME = 'Scikit Learn V1.15 %s' % (YOUR_NAME) # change to your experiment name
PROJECT_NAME =  'erwinh-mldemo' # change this to your project name

In [5]:
experiment = client.create_experiment(EXPERIMENT_NAME)

In [8]:
#TODO change to KFP github
component_url_prefix = 'https://raw.githubusercontent.com/Huize501/kfp/master/scikit-component/'

In [9]:
dummy_op = kfp.components.load_component_from_url(component_url_prefix + 'component.yaml') 

In [10]:
# Define a pipeline and create a task from a component:
@kfp.dsl.pipeline(name='KFP Scikit-learn', description='sklearn pipeline')
def my_pipeline(gcs_path_data = dsl.PipelineParam(name='pathdata', value='gs://erwinh-public-data/scikit/data/scikit-marketing-data.csv'),
               data_storage = dsl.PipelineParam(name='storage', value='GCS'),
               gcs_path_model = dsl.PipelineParam(name='pathoutput', value='gs://erwinh-public-data/kfp'), # change to your gcs path
               bigquery_table = dsl.PipelineParam(name='bqtable', value='kfp-primer-workshop.marketing_data.raw') # change to your gcs path
               ):
    dummy1_task = dummy_op(
        # Input name "Input 1 URI" is converted to pythonic parameter name "input_1_uri"
        pathdata = gcs_path_data,
        storage = data_storage,
        pathoutput =gcs_path_model,
        bqtable = bigquery_table,
        ).apply(kfp.gcp.use_gcp_secret('user-gcp-sa'))

In [11]:
pipeline_func = my_pipeline  
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'
compiler.Compiler().compile(pipeline_func, pipeline_filename)

/Users/erwinh/anaconda/envs/dlnd/lib/python3.6/site-packages/kfp/dsl/_metadata.py:118: UserWarning: Explicit creation of `kfp.dsl.PipelineParam`s by the users is deprecated. The users should define the parameter type and default values using standard pythonic constructs: def my_func(a: int = 1, b: str = "default"):
  warnings.warn('Explicit creation of `kfp.dsl.PipelineParam`s by the users is deprecated. The users should define the parameter type and default values using standard pythonic constructs: def my_func(a: int = 1, b: str = "default"):')


In [27]:
#Specify pipeline argument values
arguments = {}

#Submit a pipeline run
run_name = pipeline_func.__name__ + ' run %s' %datetime.now()
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)